Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

---

Please **submit this Jupyter notebook through Canvas** no later than **Monday November 23 12:59**, before the start of the lecture.

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Write down the names + student ID of the people in your group.

Daniël Vink - 10675140 \
Sam Kuilboer - 12442690

In [1]:
# Evaluate this cell to import necessary packages
import numpy as np
import matplotlib.pyplot as plt
import math

# Sparse matrices
A _sparse matrix_ or _sparse array_ is a matrix in which most of the elements are zero. There is no strict definition how many elements need to be zero for a matrix to be considered sparse but a common criterion is that the number of non-zero elements is roughly the number of rows or columns. By contrast, if most of the elements are nonzero, then the matrix is considered _dense_. 

In the context of software for scientific computing, a sparse matrix typically refers to a storage format, in which elements which are known to be zero are not stored. In Python, the library `scipy.sparse` defines several sparse matrix classes, such as `scipy.sparse.coo_matrix`. To construct such an object, one passes for each nonzero element the value, and the row and column coordinates. In some cases, one can also just pass the nonzero (off-)diagonals, see `scipy.sparse.diags`.

Functions for dense matrices do not always work with sparse matrices. For example for the product of a sparse matrix with a (dense) vector, there is the member function `scipy.sparse.coo_matrix.dot`, and for solving linear equations involving a sparse matrix, there is the function `scipy.sparse.linalg.spsolve`.

In [2]:
from scipy.sparse import coo_matrix

# This is how to create a sparse matrix from a given list of (row, column, value) tuples.
row  = [0,   3,   1,   0]
col  = [0,   3,   1,   2]
data = [4.0, 5.0, 7.0, 9.0]
M = coo_matrix((data, (row, col)), shape=(4, 4))

print("When printing a sparse matrix, it shows its nonzero entries:")
print(M)

print("If you want to see its `dense` matrix form, you have to use `mat.toarray()`:")
print(M.toarray())

# This is how to perform matrix-vector products.
x = np.array([1, 2, 3, 4])
print("For x={}, Mx = {}".format(x, M.dot(x)))

When printing a sparse matrix, it shows its nonzero entries:
  (0, 0)	4.0
  (3, 3)	5.0
  (1, 1)	7.0
  (0, 2)	9.0
If you want to see its `dense` matrix form, you have to use `mat.toarray()`:
[[4. 0. 9. 0.]
 [0. 7. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 5.]]
For x=[1 2 3 4], Mx = [31. 14.  0. 20.]


This matrix has values on its diagonal and on offdiagonals 1 and 2 rows ABOVE it.
[[1. 1. 1. 0.]
 [0. 2. 2. 2.]
 [0. 0. 3. 3.]
 [0. 0. 0. 4.]]
This matrix has values on its diagonal and on offdiagonals 1 and 2 rows BELOW it.
[[1. 0. 0. 0.]
 [1. 2. 0. 0.]
 [1. 2. 3. 0.]
 [0. 2. 3. 4.]]
If you want to visualize the matrix for yourself, use `plt.imshow`:


For b=[1 2 3 4], the solution x to Mx=b is [1.         0.5        0.33333333 0.5       ]
And indeed, Mx - b = [0. 0. 0. 0.]


In [3]:
from scipy.sparse import diags, SparseEfficiencyWarning
from scipy.sparse.linalg import spsolve
import warnings
warnings.simplefilter('ignore', SparseEfficiencyWarning)  # Suppress confusing warning

# This is how to create a sparse matrix from a given list of subdiagonals.
diagonals = [[1, 2, 3, 4], [1, 2, 3], [1, 2]]
M = diags(diagonals, [0, 1, 2])
print("This matrix has values on its diagonal and on offdiagonals 1 and 2 rows ABOVE it.")
print(M.toarray())

M = diags(diagonals, [0, -1, -2])
print("This matrix has values on its diagonal and on offdiagonals 1 and 2 rows BELOW it.")
print(M.toarray())

print("If you want to visualize the matrix for yourself, use `plt.imshow`:")
plt.imshow(M.toarray())
plt.colorbar()
plt.show()

# This is how to solve sparse systems.
b = np.array([1, 2, 3, 4])
x = spsolve(M, b)
print("For b={}, the solution x to Mx=b is {}".format(b, x))
print("And indeed, Mx - b = {}".format(M.dot(x) - b))

--------
# Exercise 1
Consider the following boundary value problem involving a nonlinear ordinary differential equation:

$$ \tag{1} y''(x) + \exp(y(x)) = 0, \quad 0 < x < 1, \quad y(0) = y(1) = 0.$$

The purpose of this exercise is to approximate the solution to this boundary value problem, by discretizing the problem and then solving the resulting system of nonlinear equations. 

Problem (1) will be discretized using finite differences. Suppose we use $n+2$ discretization points for $x$, denoted $x_k = kh$ for $k \in \{0, \ldots, n+1\}$ and $h = 1/(n+1)$. The approximate solution is denoted $y_k = y(x_k)$. 

We will use a _second-order central finite difference_ approximation for the second derivative: 

$$ \tag{2} y''(x_k) \approx \frac{y_{k-1} - 2 y_k + y_{k+1}}{h^2}.$$

The term $\exp(y(x_k))$ can simply be approximated by $\exp(y_k)$. Thus for $x=x_k$, equation (1) becomes

$$ \tag{3} \frac{y_{k-1} - 2 y_k + y_{k+1}}{h^2} + \exp y_k = 0, \quad k = 1, \ldots, n.$$

The boundary conditions (the conditions $y(0) = y(1) = 1$), lead to the requirement that $y_0 = y_{n+1} = 0$. To find the remaining values $y_k$, $k=1, \ldots, n$, equation (3) will be used for $k = 1, \ldots, n$. In this way, one obtains $n$ equations for $n$ unknowns, to which, in principle, a rootfinding method can be applied.

We will write $\vec y = \begin{bmatrix} y_1 \\ \vdots \\ y_n \end{bmatrix}$ for the vector of values to be determined. 


## (a) -- 2 points
As a first step, finish the function `SecondDerMatrix` that returns a matrix $\bf{M}$ that maps the vector $\vec y$ to the vector of the approximate values $y''(x_k)$, $k=1, \ldots, n$ given in (2). To get full points for this part of the exercise you must create output in the form of a sparse matrix.

In [62]:
from scipy.sparse import coo_matrix

''' We can rewrite the requested matrix as a system of equations of the form eq.(2):
    (1*yk-1 - 2*yk + 1*yk+1) / h^2 = y''k
    In matrix form, this can be seen as -2/h^2 on the main diagonal, and 1/h^2 on the diagonals directly
    above and below. Because we know y0 = yn+1 = 0, we can leave these out of the of the equation entirely.
'''

def SecondDerMatrix(n):
    ''' Returns the matrix M_n that maps a vector y of length n to their approximated values.
    
    These approximated values are a second-order central finite difference approximation of the second derivative.
    The matrix consists of a -2 coefficient main diagonal, and a 1 sub diagonal directly above and below the main
    diagonal. The coeffecients are multiplied by h^2, where h = 1/(n+1).
    '''
    h2 = (1 / (n+1))**2
    
    main_diag = [-2/h2]*n
    sub_diag  = [1/h2] *(n-1)# First sub diagonal pf a nxn matrix is size n-1
#     main_diag = [-2]*n
#     sub_diag  = [1]   *(n-1)
    diagonals = [sub_diag, main_diag, sub_diag]
    
    M = diags(diagonals, [-1, 0, 1])
    return M

M3 = SecondDerMatrix(3)
print(M3.toarray())
b = np.array([1, 2, 3])
x = spsolve(M3, b)
print(b, x)
print(M3.dot(x) - b)

[[-32.  16.   0.]
 [ 16. -32.  16.]
 [  0.  16. -32.]]
[1 2 3] [-0.15625 -0.25    -0.21875]
[0. 0. 0.]


## (b) -- 1 point
Second-order central finite differences are exact for quadratic functions. In order to test your implementation, choose $n=10$ and apply the second derivative matrix from part (a) to a quadratic function $y(x)$ with $y(0) = y(1) = 0$ for which you know the second derivative $y''(x)$.

In [66]:
print('''
For a quadratic function that has values in a range between [0 < x < 1] we decided on a simple 
inverted hyperbola which passes through the x-axis twice at x=0 and x=1. This means its "peak" will
be positive numbers for the range between 0 and 1. The inverted hyperbola will have to be shifted
to the right and upwards with respect to the origin. The function thus has the form:

f(x) = -(x-0.5)^2 + 0.5
which can be rewritten as:
f(x) = -x^2 + x

For which the second derivative f''(x) can be derived as:
f'(x) = -2x + 1
f''(x) = -2

''')

def our_hyperbola(x):
    y = -(x**2) + x
    return y

print('Function test for the peak and intersection points with the x-axis: ', 
      our_hyperbola(np.array([0, 0.5, 1])), '\n')

M10 = SecondDerMatrix(10)
x = np.linspace(0, 1, 12)[1:-1] # X divided in n+2 sections, cut off 0 and 1.
y = our_hyperbola(x)

print('Approximated (exact) solution to My = y2: ', M10.dot(y))



For a quadratic function that has values in a range between [0 < x < 1] we decided on a simple 
inverted hyperbola which passes through the x-axis twice at x=0 and x=1. This means its "peak" will
be positive numbers for the range between 0 and 1. The inverted hyperbola will have to be shifted
to the right and upwards with respect to the origin. The function thus has the form:

f(x) = -(x-0.5)^2 + 0.5
which can be rewritten as:
f(x) = -x^2 + x

For which the second derivative f''(x) can be derived as:
f'(x) = -2x + 1
f''(x) = -2


Function test for the peak and intersection points with the x-axis:  [0.   0.25 0.  ] 

Approximated (exact) solution to My = y2:  [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]


## (c) -- 2 points


Defining $\vec y = \begin{bmatrix} y_1 \\ \vdots \\ y_n \end{bmatrix}$ and 
$E(\vec y) = \begin{bmatrix} \exp(y_1) \\ \vdots \\ \exp(y_n) \end{bmatrix}$, the equations (3) can be written in the form

$$ F(\vec y) := {\bf M} \cdot \vec y + E(\vec y) = \vec 0. $$

Finish the function `F` that defines $F(\vec y) = {\bf M} \cdot \vec y + E(\vec y)$.
Finish the function `JacobianF` that computes the Jacobian ${\bf J}_F(\vec y)$ of $F(\vec y)$.
To get full points for this part of the exercise, the Jacobian must be computed in the form of a sparse matrix.

In [ ]:
def F(y):
    # YOUR CODE HERE
    raise NotImplementedError()
    
def JacobianF(y):
    # YOUR CODE HERE
    raise NotImplementedError()

## (d) -- 3 points
1. Write down the first order Taylor expansion $T_F(\vec y, \vec s)$ for $F(\vec y+\vec s)$.
2. In order to check your implementation of the Jacobian matrix, compute and print both $F(\vec y+\vec s)$ and its first order Taylor approximation $T_F(\vec y, \vec s)$ for a choice $\vec y$ and $\vec s$.
3. Verify numerically that the error $||F(\vec y + \vec s) - T_F(\vec y, \vec s)||_2$ is $\mathcal O(||\vec s||_2^2)$. Hint: take vectors $\vec s$ with $\|\vec s\|_2 = \mathcal O(h)$  for multiple values for $h$, e.g. $h = 10^{-k}$ for a range of $k$. 

Subquestion 1.

YOUR ANSWER HERE

In [ ]:
# Subquestions 2 and 3.

# YOUR CODE HERE
raise NotImplementedError()

## (e) -- 2 points
1. Finish the function `NewtonSolve` below to solve the system of equations.
2. Take $n=40$, and experiment with your function. Try to find a choice of `y0` such that the method doesn't converge, as well as a choice of `y0` such that the method converges. In your answer, list the types of convergence behavior you found. Show a convergent example (if you found any) and a nonconvergent example (if you found any). Show the solutions you found for each example.


In [ ]:
# Subquestion 1.
def NewtonSolve(y0, K):
    """ Use Newton's method to solve F(y) = 0 with initial guess y0 and K iterations. """
    y = y0
    # YOUR CODE HERE
    raise NotImplementedError()
    return y

In [ ]:
# Subquestion 2, code part
# YOUR CODE HERE
raise NotImplementedError()

Subquestion 2, text part
YOUR ANSWER HERE